In [1]:
import numpy as np 
import pandas as pd 
import os

import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models

print("You have TensorFlow version", tf.__version__)

C:\Users\DOCTOR\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\DOCTOR\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\DOCTOR\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\DOCTOR\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

You have TensorFlow version 1.14.0


In [2]:
data = pd.read_csv(r"C:\Users\DOCTOR\Downloads\bbc-text.csv")

In [3]:
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 1780
Test size: 445


In [4]:
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

In [5]:
train_cat, test_cat = train_test_split(data['category'], train_size)
train_text, test_text = train_test_split(data['text'], train_size)

In [6]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words,char_level=False)

In [7]:
tokenize.fit_on_texts(train_text) 
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [8]:
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)

In [9]:
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [10]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (1780, 1000)
x_test shape: (445, 1000)
y_train shape: (1780, 5)
y_test shape: (445, 5)


In [11]:
batch_size = 32
epochs = 10
drop_ratio = 0.5

In [12]:
model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
# model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 1602 samples, validate on 178 samples
Epoch 1/10
1602/1602 [==============================] - 1s 356us/sample - loss: 0.4672 - acc: 0.8645 - val_loss: 0.1564 - val_acc: 0.9382
Epoch 2/10
1602/1602 [==============================] - 0s 256us/sample - loss: 0.0444 - acc: 0.9906 - val_loss: 0.1216 - val_acc: 0.9551
Epoch 3/10
1602/1602 [==============================] - 0s 249us/sample - loss: 0.0123 - acc: 1.0000 - val_loss: 0.1107 - val_acc: 0.9551
Epoch 4/10
1602/1602 [==============================] - 0s 249us/sample - loss: 0.0076 - acc: 1.0000 - val_loss: 0.1130 - val_acc: 0.9551
Epoch 5/10
1602/1602 [==============================] - 0s 251us/sample - loss: 0.0042 - acc: 1.0000 - val_loss: 0.1136 - val_acc: 0.9551
Epoch 6/10
1602/1602 [==============================] - 0s 264us/sample - loss: 0.0030 - acc: 1.0000 - val_loss: 0.1168 - val_acc: 0.9551
Epoch 7/10
1602/1602 [==============================] - 0s 262us/sample - loss: 0.0022 - acc: 1.0000 - val_loss: 0.1187 - val

In [14]:
score = model.evaluate(x_test, y_test,batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

445/445 [==============================] - 0s 83us/sample - loss: 0.1267 - acc: 0.9618
Test loss: 0.1266817765289478
Test accuracy: 0.9617978
